# Arnheim 3 - Segmented Patch Creator
DeepMind, 2021

## Intructions
This Colab is to support the creation of segmented patches for collage creation using Arnheim 3.

The Colab uses [PixelLib](https://github.com/ayoolaolafenwa/PixelLib) and is pretty basic but good enough to get one started creating patches from JPG images.

The process is

1) Provide source images

* Upload images using this Colab to either Google Drive or the temp folder
* Alternatively use a Google Drive folder that already contains images

2) Create segmented patches
* The patch file is save to Google Drive. Be sure to copy the location of the file in the Arnheim 3 Colab.




In [ ]:
#@title Installations
!pip3 install pixellib
!pip3 install tensorflow==2.0.1
!pip3 install Keras==2.3.0
!pip3 install h5py==2.10.0

In [ ]:
#@title Imports
import glob
from google.colab import drive
from google.colab import files
import io
import numpy as np
import os
import pathlib
import pixellib
from pixellib.instance import instance_segmentation
import requests


In [ ]:
#@title Function definitions

def mkdir(path):
  pathlib.Path(path).mkdir(parents=True, exist_ok=True)

def upload_files(target_path):
  """Upload files to target directory."""
  mkdir(target_path)
  uploaded = files.upload()
  for k, v in uploaded.items():
    open(target_path + "/" + k, 'wb').write(v)
  return list(uploaded.keys())


def download_from_url(url, force=False):
  """Download file from URL and cache it."""

  cache_dir = "/content/cache"
  mkdir(cache_dir)
  cache_filename = f"{cache_dir}/{os.path.basename(url)}"
  cache = pathlib.Path(cache_filename)
  if not cache.is_file() or force:
    print("Downloading " + url)
    r = requests.get(url)
    bytesio_object = io.BytesIO(r.content)
    with open(cache_filename, "wb") as f:
        f.write(bytesio_object.getbuffer())
  else:
    print("Using cached version of " + url)
  return cache 

In [ ]:
#@title Authorise and mount Google Drive
ROOT = "/content"
MOUNT_DIR = f"{ROOT}/drive"
drive.mount(MOUNT_DIR)
# ROOT_PATH = f"{MOUNT_DIR}/MyDrive/Arnheim3"
# 
# mkdir(ROOT)
# IMAGE_PATH = f"{ROOT}/source_images"
# SEGMENTED_PATH = f"{ROOT}/segmented"
# 
# print(f"\nUsing base directory: {ROOT}")
# print(f"Source images directory: {IMAGE_PATH}")
# print(f"Segmented directory: {SEGMENTED_PATH}")

In [ ]:
#@title Source images and target file locations
#@markdown Source images can be stored temporarily with the Colab, be already on Google Drive, or can be uploaded to Google Drive.
use_google_drive_for_source_images = True   #@param {type:"boolean"}
#@markdown Source images (if stored on Google Drive)
GOOGLE_DRIVE_PATH_SOURCE_IMAGES = "Art/Collage/Images" #@param {type:"string"}
#@markdown Target segmentation file will be saved to Google Drive for use with the Arnheim 3 Colab.
SEGMENTED_DATA_FILENAME = "fruit.npy"  #@param {type: "string"}
GOOGLE_DRIVE_PATH_SEGMENTED_DATA = "Art/Collage/Patches" #@param {type:"string"}

data_path = MOUNT_DIR + "/MyDrive/" + GOOGLE_DRIVE_PATH_SEGMENTED_DATA
data_file = data_path + "/" + SEGMENTED_DATA_FILENAME

if use_google_drive_for_source_images:
  IMAGE_PATH = MOUNT_DIR + "/MyDrive/" + GOOGLE_DRIVE_PATH_SOURCE_IMAGES
else:
  IMAGE_PATH = f"{ROOT}/source_images"
mkdir(IMAGE_PATH)
mkdir(data_path)

print(f"Source images directory: {IMAGE_PATH}")
print(f"Segmented data will be saved to: {data_file}")

In [ ]:
#@title Run this cell to upload a new set of images to segment
empty_target_dir_before_upload = False  #@param {type:"boolean"}

if empty_target_dir_before_upload:
  !rm {IMAGE_PATH}/*

upload_files(IMAGE_PATH)
print(f"Images uploaded images to {IMAGE_PATH}")

!ls -l {IMAGE_PATH}

In [ ]:
#@title Segment images and save patch file

# https://pixellib.readthedocs.io/en/latest/Image_instance.html
segment_image = instance_segmentation()
segmentation_model_file = download_from_url(
    "https://github.com/ayoolaolafenwa/PixelLib/releases/download/1.2/mask_rcnn_coco.h5")
segment_image.load_model(segmentation_model_file)

imagefiles = []
for file in glob.glob(f"{IMAGE_PATH}/*.jpg"):
  imagefiles.append(file)

print(imagefiles)
print("num images to process = ", len(imagefiles))

segmented_images = []
for imagefile in imagefiles:
  print(imagefile)
  try:
    seg, _ = segment_image.segmentImage(
        imagefile,
        extract_segmented_objects=True,
        save_extracted_objects =False,
        show_bboxes=False,
        output_image_name=str(imagefile) + "______.tiff")
  except:
    print("Error encounted - skipping")
    continue

  if not len(seg["extracted_objects"]):
    print("Failed to segment", imagefile)
  else:
    for result in seg["extracted_objects"]:
      print(result.shape)
      segmented_image = result[..., ::-1].copy()
      segmented_images.append(segmented_image)

with open(data_file, "wb") as f:
  np.save(f, segmented_images)
print("Saved patch file to", data_file)